# REAL-TIME WEED DETECTION SYSTEM

In [ ]:
# Real-Time Weed Detection System - SMART VERSION
# Auto-detects Colab vs Local, handles paths correctly!

# ============================================
# INSTALLATION (Uncomment and run first time)
# ============================================
#!pip install ultralytics opencv-python pillow matplotlib ipywidgets torch torchvision pyyaml -q

# ============================================
# ENVIRONMENT DETECTION
# ============================================

In [ ]:

import sys
import os

def detect_environment():
    """Detect if running in Colab or local"""
    try:
        import google.colab
        return 'colab'
    except:
        return 'local'

ENV = detect_environment()

# ============================================
# IMPORTS
# ============================================
import cv2
import torch
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets
from io import BytesIO
from ultralytics import YOLO
import yaml
import time

try:
    plt.style.use('seaborn-v0_8-darkgrid')
except:
    plt.style.use('default')

print("✅ All libraries imported successfully!")
print(f"🖥️ Environment: {ENV.upper()}")

# ============================================
# CHECK FOR EXISTING TRAINED MODEL
# ============================================
print("\n" + "="*70)
print("🔍 CHECKING FOR TRAINED MODEL")
print("="*70)

# Look for already trained model
MODEL_PATHS = [
    "runs/detect/weed_detection/weights/best.pt",
    "runs/detect/weed_detection2/weights/best.pt",
    "runs/detect/weed_detection3/weights/best.pt",
    "weed_model.pt"
]

trained_model = None
for path in MODEL_PATHS:
    if os.path.exists(path):
        print(f"✅ Found trained model: {path}")
        trained_model = YOLO(path)
        print("✅ Model loaded successfully!")
        break

# ============================================
# TRAINING SECTION (Only if no model found)
# ============================================
if trained_model is None:
    print("\n❌ No trained model found!")

    if ENV == 'colab':
        print("\n🌐 You're in GOOGLE COLAB")
        print("="*70)
        print("⚠️ To train the model, you need to upload your dataset to Colab.")
        print("\nYou have 3 options:")
        print("  1. Skip training and use pre-trained YOLOv8 (won't detect weeds accurately)")
        print("  2. Upload dataset to Colab now")
        print("  3. Mount Google Drive (if dataset is in Drive)")
        print("="*70)

        choice = input("\nEnter choice (1/2/3): ").strip()

        if choice == '1':
            print("\n⚠️ Using base YOLOv8n model (NOT trained for weed detection)")
            print("NOTE: Results won't be accurate for weeds!")
            trained_model = YOLO('yolov8n.pt')

        elif choice == '3':
            # Mount Google Drive
            from google.colab import drive
            drive.mount('/content/drive')
            print("\n✅ Google Drive mounted at /content/drive/")
            print("\nExample paths:")
            print("  /content/drive/MyDrive/Weed Detection")
            print("  /content/drive/MyDrive/datasets/WeedDetection")

            BASE_PATH = input("\n📁 Enter path to your dataset folder: ").strip()
            BASE_PATH = BASE_PATH.replace('"', '').replace("'", "")

            if os.path.exists(BASE_PATH):
                print(f"✅ Found: {BASE_PATH}")
                train_now = 'y'
            else:
                print(f"❌ Path not found: {BASE_PATH}")
                print("Using base model instead...")
                trained_model = YOLO('yolov8n.pt')
                train_now = 'n'

        else:  # choice == '2'
            print("\n📤 UPLOAD YOUR DATASET:")
            print("1. Click 📁 Files icon on left sidebar")
            print("2. Upload your 'Weed Detection' folder (or zip file)")
            print("3. If uploading zip, extract it after upload")
            print("4. Come back here after upload completes")

            input("\n⏸️ Press Enter when upload is done...")

            # Look for common paths
            possible = ['/content/Weed Detection', '/content/WeedDetection',
                       '/content/weed_detection', '/content/dataset']

            BASE_PATH = None
            for p in possible:
                if os.path.exists(p):
                    BASE_PATH = p
                    print(f"✅ Found dataset at: {BASE_PATH}")
                    break

            if not BASE_PATH:
                BASE_PATH = input("\n📁 Enter the full path where you uploaded: ").strip()
                BASE_PATH = BASE_PATH.replace('"', '').replace("'", "")

            if os.path.exists(BASE_PATH):
                train_now = 'y'
            else:
                print(f"❌ Path not found: {BASE_PATH}")
                print("Using base model...")
                trained_model = YOLO('yolov8n.pt')
                train_now = 'n'

    else:  # Local environment
        print("\n💻 You're on LOCAL COMPUTER")
        print("="*70)
        train_now = input("Do you want to train now? (y/n): ").strip().lower()

        if train_now == 'y':
            BASE_PATH = r"D:\Weed Detection"

            if not os.path.exists(BASE_PATH):
                print(f"\n⚠️ Default path not found: {BASE_PATH}")
                print("\nEnter the full path (without quotes):")
                print("Example: D:\\Weed Detection")
                print("     or: C:\\Users\\YourName\\Documents\\Weed Detection")

                BASE_PATH = input("\n📁 Path: ").strip()
                # Remove quotes if user added them
                BASE_PATH = BASE_PATH.replace('"', '').replace("'", "")

            if not os.path.exists(BASE_PATH):
                print(f"\n❌ Path still not found: {BASE_PATH}")
                print("\n💡 Troubleshooting:")
                print("  • Make sure folder exists at that location")
                print("  • Check spelling (case-sensitive on some systems)")
                print("  • Don't include quotes in the path")
                print("  • Use double backslashes: D:\\\\Weed Detection")
                print("\nUsing base model for now...")
                trained_model = YOLO('yolov8n.pt')
                train_now = 'n'
        else:
            print("\n⚠️ Using base YOLOv8n model")
            trained_model = YOLO('yolov8n.pt')

    # ============================================
    # ACTUAL TRAINING
    # ============================================
    if train_now == 'y' and trained_model is None:
        print(f"\n✅ Dataset location: {BASE_PATH}")

        # Setup paths
        TRAIN_IMAGES = os.path.join(BASE_PATH, "images", "train")
        VAL_IMAGES = os.path.join(BASE_PATH, "images", "val")
        TEST_IMAGES = os.path.join(BASE_PATH, "images", "test")
        TRAIN_LABELS = os.path.join(BASE_PATH, "labels", "train")
        VAL_LABELS = os.path.join(BASE_PATH, "labels", "val")
        TEST_LABELS = os.path.join(BASE_PATH, "labels", "test")
        CLASSES_FILE = os.path.join(BASE_PATH, "classes.txt")

        # Verify folders
        print("\n🔍 Verifying dataset structure...")
        print("-"*70)

        required = {
            "Train Images": TRAIN_IMAGES,
            "Val Images": VAL_IMAGES,
            "Train Labels": TRAIN_LABELS,
            "Val Labels": VAL_LABELS
        }

        all_ok = True
        for name, path in required.items():
            if os.path.exists(path):
                try:
                    count = len([f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))])
                    print(f"✅ {name:15s}: {count:4d} files")
                except:
                    print(f"⚠️ {name:15s}: Error reading")
                    all_ok = False
            else:
                print(f"❌ {name:15s}: NOT FOUND")
                all_ok = False

        print("-"*70)

        if not all_ok:
            print("\n❌ Dataset structure incomplete!")
            print("\n📋 Required structure:")
            print("Weed Detection/")
            print("  ├── images/train/  (training images)")
            print("  ├── images/val/    (validation images)")
            print("  ├── labels/train/  (training labels)")
            print("  └── labels/val/    (validation labels)")
            print("\nUsing base model instead...")
            trained_model = YOLO('yolov8n.pt')
        else:
            # Read or create classes
            if os.path.exists(CLASSES_FILE):
                with open(CLASSES_FILE, 'r', encoding='utf-8') as f:
                    classes = [line.strip() for line in f.readlines() if line.strip()]
            else:
                print("⚠️ classes.txt not found, creating default...")
                classes = ['crop', 'weed']
                try:
                    with open(CLASSES_FILE, 'w', encoding='utf-8') as f:
                        f.write('\n'.join(classes))
                except:
                    pass

            print(f"✅ Classes: {classes}")

            # Create YAML
            PROJECT_DIR = "weed_detection_project"
            os.makedirs(PROJECT_DIR, exist_ok=True)

            yaml_content = {
                'path': os.path.abspath(BASE_PATH),
                'train': os.path.abspath(TRAIN_IMAGES),
                'val': os.path.abspath(VAL_IMAGES),
                'test': os.path.abspath(TEST_IMAGES) if os.path.exists(TEST_IMAGES) else os.path.abspath(VAL_IMAGES),
                'nc': len(classes),
                'names': classes
            }

            yaml_path = os.path.join(PROJECT_DIR, 'weed_data.yaml')
            with open(yaml_path, 'w', encoding='utf-8') as f:
                yaml.dump(yaml_content, f, default_flow_style=False)

            print(f"✅ YAML config created")

            # Train
            print("\n" + "="*70)
            print("🚀 STARTING TRAINING")
            print("="*70)

            device = 'cuda' if torch.cuda.is_available() else 'cpu'
            print(f"🖥️ Device: {device.upper()}")

            if device == 'cpu':
                print("⚠️ Training on CPU (slow). Consider using GPU.")

            epochs_input = input("\nEnter epochs (default 50, recommend 30-100): ").strip()
            epochs = int(epochs_input) if epochs_input.isdigit() else 50

            batch = 4 if device == 'cpu' else 16

            print(f"\n🏋️ Training: {epochs} epochs, batch {batch}, 640px")
            print("This will take time. Please wait...")

            model = YOLO('yolov8n.pt')

            try:
                results = model.train(
                    data=yaml_path,
                    epochs=epochs,
                    imgsz=640,
                    batch=batch,
                    name='weed_detection',
                    patience=15,
                    save=True,
                    plots=True,
                    device=device,
                    exist_ok=True,
                    verbose=True
                )

                print("\n✅ Training complete!")
                trained_model = model

                # Save to easy location
                model.save('weed_model.pt')
                print("✅ Model saved as 'weed_model.pt'")
                print("Next time you run this, it will load automatically!")

            except Exception as e:
                print(f"\n❌ Training failed: {e}")
                print("Using base model...")
                trained_model = YOLO('yolov8n.pt')

# Set final model
if trained_model is None:
    trained_model = YOLO('yolov8n.pt')

model = trained_model

print("\n" + "="*70)
print("✅ MODEL READY!")
print("="*70)

# ============================================
# DASHBOARD STYLING
# ============================================
dashboard_style = """
<style>
    .dashboard-container {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        padding: 30px;
        border-radius: 20px;
        box-shadow: 0 10px 40px rgba(0,0,0,0.3);
        margin: 20px 0;
    }
    .dashboard-title {
        color: white;
        font-size: 42px;
        font-weight: bold;
        text-align: center;
        margin-bottom: 10px;
        text-shadow: 2px 2px 4px rgba(0,0,0,0.3);
    }
    .dashboard-subtitle {
        color: #e0e7ff;
        font-size: 18px;
        text-align: center;
        margin-bottom: 30px;
    }
    .feature-title {
        color: #667eea;
        font-size: 24px;
        font-weight: bold;
        margin-bottom: 15px;
        padding: 10px;
        background: #f0f4ff;
        border-radius: 10px;
    }
    .info-box {
        background: #e0f2fe;
        border-left: 4px solid #0284c7;
        padding: 15px;
        margin: 10px 0;
        border-radius: 5px;
    }
</style>
"""

display(HTML(dashboard_style))

# ============================================
# DASHBOARD
# ============================================
class WeedDetectionDashboard:
    def __init__(self, model):
        self.model = model
        self.webcam_running = False
        self.setup_ui()

    def setup_ui(self):
        """Setup dashboard UI"""

        header_html = """
        <div class="dashboard-container">
            <div class="dashboard-title">🌿 Real-Time Weed Detection System</div>
            <div class="dashboard-subtitle">Upload Images • Upload Videos • Use Live Webcam</div>
        </div>
        <div class="info-box">
            <strong>📖 How to use:</strong> Simply upload any image or video to detect weeds and crops!
        </div>
        """
        display(HTML(header_html))

        # Tab 1: Image
        self.image_upload = widgets.FileUpload(accept='image/*', multiple=False)
        self.image_output = widgets.Output()
        self.image_btn = widgets.Button(
            description='🔍 Detect Weeds',
            button_style='primary',
            layout=widgets.Layout(width='200px', height='50px')
        )
        self.image_btn.on_click(self.detect_image)

        image_tab = widgets.VBox([
            widgets.HTML("<div class='feature-title'>📷 Image Detection</div>"),
            self.image_upload,
            self.image_btn,
            self.image_output
        ])

        # Tab 2: Video
        self.video_upload = widgets.FileUpload(accept='video/*', multiple=False)
        self.video_output = widgets.Output()
        self.video_btn = widgets.Button(
            description='🎥 Process Video',
            button_style='primary',
            layout=widgets.Layout(width='200px', height='50px')
        )
        self.video_btn.on_click(self.detect_video)

        video_tab = widgets.VBox([
            widgets.HTML("<div class='feature-title'>🎥 Video Detection</div>"),
            self.video_upload,
            self.video_btn,
            self.video_output
        ])

        # Tab 3: Webcam
        if ENV == 'colab':
            webcam_tab = widgets.VBox([
                widgets.HTML("<div class='feature-title'>📹 Webcam</div>"),
                widgets.HTML("<p>⚠️ Webcam not available in Colab. Use Image/Video upload.</p>")
            ])
        else:
            self.webcam_output = widgets.Output()
            self.webcam_start = widgets.Button(
                description='📹 Start',
                button_style='success',
                layout=widgets.Layout(width='150px', height='50px')
            )
            self.webcam_stop = widgets.Button(
                description='⏹️ Stop',
                button_style='danger',
                layout=widgets.Layout(width='150px', height='50px'),
                disabled=True
            )
            self.webcam_start.on_click(self.start_webcam)
            self.webcam_stop.on_click(self.stop_webcam)

            webcam_tab = widgets.VBox([
                widgets.HTML("<div class='feature-title'>📹 Live Webcam</div>"),
                widgets.HBox([self.webcam_start, self.webcam_stop]),
                self.webcam_output
            ])

        self.tab = widgets.Tab()
        self.tab.children = [image_tab, video_tab, webcam_tab]
        self.tab.set_title(0, '📷 Image')
        self.tab.set_title(1, '🎥 Video')
        self.tab.set_title(2, '📹 Webcam')

        display(self.tab)

    def detect_image(self, btn):
        self.image_output.clear_output()

        with self.image_output:
            if not self.image_upload.value:
                print("⚠️ Please upload an image!")
                return

            try:
                print("🔄 Processing...")

                uploaded = list(self.image_upload.value.values())[0]
                image = Image.open(BytesIO(uploaded['content']))

                if image.mode == 'RGBA':
                    image = image.convert('RGB')

                results = self.model(image, verbose=False)

                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

                ax1.imshow(image)
                ax1.set_title('Original', fontsize=14, fontweight='bold')
                ax1.axis('off')

                result_img = results[0].plot()
                ax2.imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
                ax2.set_title('Detections', fontsize=14, fontweight='bold')
                ax2.axis('off')

                plt.tight_layout()
                plt.show()

                boxes = results[0].boxes
                if len(boxes) > 0:
                    crops = sum(1 for c in boxes.cls if int(c) == 0)
                    weeds = sum(1 for c in boxes.cls if int(c) == 1)

                    print("\n" + "="*60)
                    print("📊 RESULTS")
                    print("="*60)
                    print(f"✅ Crops: {crops}")
                    print(f"🌿 Weeds: {weeds}")
                    print(f"📍 Total: {len(boxes)}")
                    print(f"🎯 Confidence: {boxes.conf.mean():.2f}")
                    print("="*60)
                else:
                    print("\n⚠️ No detections")

            except Exception as e:
                print(f"❌ Error: {e}")

    def detect_video(self, btn):
        self.video_output.clear_output()

        with self.video_output:
            if not self.video_upload.value:
                print("⚠️ Please upload a video!")
                return

            try:
                print("🔄 Processing video...")

                uploaded = list(self.video_upload.value.values())[0]
                video_path = "temp.mp4"

                with open(video_path, 'wb') as f:
                    f.write(uploaded['content'])

                output = "detected.mp4"

                cap = cv2.VideoCapture(video_path)
                if not cap.isOpened():
                    print("❌ Cannot open video")
                    return

                fps = int(cap.get(cv2.CAP_PROP_FPS))
                w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

                print(f"📊 {w}x{h} @ {fps}fps, {total} frames")

                fourcc = cv2.VideoWriter_fourcc(*'mp4v')
                out = cv2.VideoWriter(output, fourcc, fps, (w, h))

                frame_num = 0
                total_crops = 0
                total_weeds = 0

                while True:
                    ret, frame = cap.read()
                    if not ret:
                        break

                    results = self.model(frame, verbose=False)
                    annotated = results[0].plot()
                    out.write(annotated)

                    if len(results[0].boxes) > 0:
                        total_crops += sum(1 for c in results[0].boxes.cls if int(c) == 0)
                        total_weeds += sum(1 for c in results[0].boxes.cls if int(c) == 1)

                    frame_num += 1
                    if frame_num % 30 == 0:
                        pct = (frame_num/total)*100 if total > 0 else 0
                        print(f"⏳ {pct:.1f}% ({frame_num}/{total})")

                cap.release()
                out.release()

                print("\n" + "="*60)
                print("✅ COMPLETE")
                print("="*60)
                print(f"📁 {output}")
                print(f"🎬 {frame_num} frames")
                print(f"✅ Crops: {total_crops}")
                print(f"🌿 Weeds: {total_weeds}")
                print("="*60)

                if ENV == 'colab':
                    from google.colab import files
                    files.download(output)

                try:
                    os.remove(video_path)
                except:
                    pass

            except Exception as e:
                print(f"❌ Error: {e}")

    def start_webcam(self, btn):
        self.webcam_output.clear_output()
        self.webcam_running = True
        self.webcam_start.disabled = True
        self.webcam_stop.disabled = False

        with self.webcam_output:
            try:
                cap = cv2.VideoCapture(0)

                if not cap.isOpened():
                    print("❌ Cannot open webcam")
                    self.webcam_start.disabled = False
                    self.webcam_stop.disabled = True
                    return

                cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
                cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

                frame_num = 0
                last_time = time.time()

                while self.webcam_running:
                    ret, frame = cap.read()
                    if not ret:
                        break

                    if frame_num % 3 == 0:
                        results = self.model(frame, verbose=False)
                        annotated = results[0].plot()

                        fps = 1/(time.time()-last_time+0.001)
                        last_time = time.time()

                        clear_output(wait=True)
                        plt.figure(figsize=(12, 8))
                        plt.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
                        plt.title(f'Frame {frame_num} | FPS: {fps:.1f}', fontsize=14)
                        plt.axis('off')
                        plt.tight_layout()
                        plt.show()

                        if len(results[0].boxes) > 0:
                            crops = sum(1 for c in results[0].boxes.cls if int(c) == 0)
                            weeds = sum(1 for c in results[0].boxes.cls if int(c) == 1)
                            print(f"✅ Crops: {crops} | 🌿 Weeds: {weeds} | FPS: {fps:.1f}")

                    frame_num += 1
                    if frame_num > 10000:
                        break

                cap.release()
                plt.close('all')
                print("\n⏹️ Stopped")

            except Exception as e:
                print(f"❌ Error: {e}")
            finally:
                self.webcam_start.disabled = False
                self.webcam_stop.disabled = True

    def stop_webcam(self, btn):
        self.webcam_running = False

# ============================================
# LAUNCH
# ============================================
print("\n🚀 LAUNCHING DASHBOARD...")

dashboard = WeedDetectionDashboard(model)

print("\n✅ Ready! Upload images/videos in the tabs above!")
print("="*70)